In [150]:
library(tidyverse)
library(MuMIn)
library(caret)

In [151]:
wine_train <- read_csv("wineQualityTrain.csv")
wine_test <- read_csv("wineQualityTest.csv")


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double(),
  type = col_character()
)



── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double()

In [152]:
wine_train %>% head()
wine_train %>% summary()

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
8.9,0.38,0.40,2.20,0.068,12,28,0.99486,3.27,0.75,12.6,7,Red
7.8,0.21,0.34,11.90,0.039,55,140,0.99590,3.02,0.31,10.3,6,White
7.1,0.22,0.33,2.80,0.033,48,153,0.98990,3.15,0.38,12.7,7,White
7.1,0.17,0.40,14.55,0.047,47,156,0.99945,3.34,0.78,9.1,6,White
7.2,0.53,0.13,2.00,0.058,18,22,0.99573,3.21,0.68,9.9,6,Red
6.6,0.23,0.30,14.90,0.051,33,118,0.99835,3.04,0.54,9.0,6,White


 fixed acidity    volatile acidity  citric acid     residual sugar  
 Min.   : 4.500   Min.   :0.0800   Min.   :0.0000   Min.   : 0.700  
 1st Qu.: 6.400   1st Qu.:0.2300   1st Qu.:0.2500   1st Qu.: 1.800  
 Median : 7.000   Median :0.2900   Median :0.3100   Median : 2.800  
 Mean   : 7.261   Mean   :0.3381   Mean   :0.3217   Mean   : 5.291  
 3rd Qu.: 7.600   3rd Qu.:0.4100   3rd Qu.:0.4000   3rd Qu.: 7.925  
 Max.   :15.000   Max.   :1.3300   Max.   :0.8200   Max.   :22.600  
   chlorides       free sulfur dioxide total sulfur dioxide    density      
 Min.   :0.01200   Min.   :  1.00      Min.   :  8.0        Min.   :0.9875  
 1st Qu.:0.03800   1st Qu.: 17.00      1st Qu.: 73.0        1st Qu.:0.9923  
 Median :0.04700   Median : 29.00      Median :118.0        Median :0.9949  
 Mean   :0.05656   Mean   : 31.01      Mean   :114.2        Mean   :0.9947  
 3rd Qu.:0.06800   3rd Qu.: 42.00      3rd Qu.:156.0        3rd Qu.:0.9970  
 Max.   :0.41500   Max.   :146.50      Max.   :307.5   

In [153]:
wine_train <- wine_train %>% mutate(qual_01 = ifelse(quality >= 6, 1, 0))

In [154]:
names(wine_train)

[1] "fixed acidity"        "volatile acidity"     "citric acid"         
 [4] "residual sugar"       "chlorides"            "free sulfur dioxide" 
 [7] "total sulfur dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "type"                 "qual_01"

In [155]:
lin_summ <- summary(lm(quality ~ . -qual_01, data = wine_train))
lin_summ


Call:
lm(formula = quality ~ . - qual_01, data = wine_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.3895 -0.4477 -0.0631  0.4377  2.8194 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             1.161e+02  3.238e+01   3.586 0.000349 ***
`fixed acidity`         8.786e-02  3.371e-02   2.606 0.009256 ** 
`volatile acidity`     -1.434e+00  1.765e-01  -8.123 1.06e-15 ***
`citric acid`          -3.369e-02  1.751e-01  -0.192 0.847507    
`residual sugar`        6.743e-02  1.307e-02   5.157 2.90e-07 ***
chlorides              -6.621e-01  7.506e-01  -0.882 0.377903    
`free sulfur dioxide`   2.430e-03  1.659e-03   1.464 0.143388    
`total sulfur dioxide` -1.476e-03  7.223e-04  -2.043 0.041242 *  
density                -1.155e+02  3.278e+01  -3.524 0.000440 ***
pH                      5.190e-01  1.974e-01   2.629 0.008669 ** 
sulphates               1.026e+00  1.720e-01   5.966 3.14e-09 ***
alcohol                 2.142e-01  4.

In [164]:
k_validator <- function(df, k = 10){
    mse <- rep(0, k)
    k_folds <- nrow(df)/k
    for(i in 1:k){
        k1 <- k_folds*(i-1)+1
        k2 <- k_folds*i
        values <- !c(1:nrow(df) %in% (k1:k2))
        df_k <- df[values, ]
        
        fit <- lm(quality ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol` + `residual sugar`:`sulphates`, data = df_k)
        
        k_predicted <- predict(fit, newdata = df[!values, ])
        mse[i] <- sum((k_predicted - df[k1:k2, 'quality'])^2)
        }
    return(mean(mse))
    }

k_validator(wine_train)


[1] 67.61387

In [9]:
lin_fit1 <- lm(quality ~ `volatile acidity` + `residual sugar` + `density` + `sulphates` + `alcohol`, data = wine_train)
lin_fit2 <- lm(quality ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol`, data = wine_train)
lin_fit3 <- lm(quality ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol` + `residual sugar`:`sulphates`, data = wine_train)
lin_fit4 <- lm(quality ~ `volatile acidity`*`residual sugar` + `sulphates` + `alcohol`, data = wine_train)
lin_fit5 <- lm(quality ~ poly(`volatile acidity`, 2) + poly(`sulphates`, 2) + poly(`alcohol`, 2), data = wine_train)

In [182]:
k_lin_fit1 <- train(quality ~ `volatile acidity` + `residual sugar` + `density` + `sulphates` + `alcohol`, 
                   method = "lm", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_lin_fit2 <- train(quality ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol`, 
                   method = "lm", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_lin_fit3 <- train(quality ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol` + `residual sugar`:`sulphates`, 
                   method = "lm", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_lin_fit4 <- train(quality ~ `volatile acidity`*`residual sugar` + `sulphates` + `alcohol`, 
                   method = "lm", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_lin_fit5 <- train(quality ~ poly(`volatile acidity`, 2) + poly(`sulphates`, 2) + poly(`alcohol`, 2), 
                   method = "lm", data = wine_train, trControl = trainControl(method = "cv", number = 10))

lin_results <- 
data.frame(
t(k_lin_fit1$results),
t(k_lin_fit2$results),
t(k_lin_fit3$results),
t(k_lin_fit4$results),
t(k_lin_fit5$results))

print(lin_results)

                   X1       X1.1       X1.2       X1.3       X1.4
intercept  1.00000000 1.00000000 1.00000000 1.00000000 1.00000000
RMSE       0.71617008 0.71894807 0.71764568 0.71537558 0.71889923
Rsquared   0.30592486 0.30189377 0.30485034 0.30782913 0.30088620
MAE        0.55659633 0.55707272 0.55784990 0.55393036 0.56078680
RMSESD     0.07210726 0.04456811 0.05942239 0.07026473 0.04154695
RsquaredSD 0.07653138 0.08085559 0.07755766 0.07024064 0.04605431
MAESD      0.04794467 0.02539220 0.04318351 0.03904597 0.03183553


In [187]:
k_log_fit1 <- train(as.factor(qual_01) ~ `volatile acidity` + `residual sugar` + `density` + `sulphates` + `alcohol`, 
                   method = "glm", family = "binomial", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_log_fit2 <- train(as.factor(qual_01) ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol`, 
                   method = "glm", family = "binomial", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_log_fit3 <- train(as.factor(qual_01) ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol` + `residual sugar`:`sulphates`, 
                   method = "glm", family = "binomial", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_log_fit4 <- train(as.factor(qual_01) ~ `volatile acidity`*`residual sugar` + `sulphates` + `alcohol`, 
                   method = "glm", family = "binomial", data = wine_train, trControl = trainControl(method = "cv", number = 10))

k_log_fit5 <- train(as.factor(qual_01) ~ poly(`volatile acidity`, 2) + poly(`sulphates`, 2) + poly(`alcohol`, 2), 
                   method = "glm", family = "binomial", data = wine_train, trControl = trainControl(method = "cv", number = 10))

log_results <- 
data.frame(
t(k_log_fit1$results),
t(k_log_fit2$results),
t(k_log_fit3$results),
t(k_log_fit4$results),
t(k_log_fit5$results))

print(log_results)

                   X1       X1.1       X1.2       X1.3       X1.4
parameter        none       none       none       none       none
Accuracy    0.7584825  0.7599619  0.7722574  0.7646773  0.7654114
Kappa       0.4634139  0.4671261  0.4949555  0.4764685  0.4813549
AccuracySD 0.02442469 0.04305587 0.02841174 0.04022318 0.02618219
KappaSD    0.05523767 0.09672817 0.05813942 0.09246141 0.06277113


In [159]:
log_fit1 <- glm(qual_01 ~ `volatile acidity` + `residual sugar` + `density` + `sulphates` + `alcohol`, 
                family = 'binomial', data = wine_train)
log_fit2 <- glm(qual_01 ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol`, 
                data = wine_train, family = "binomial")
log_fit3 <- glm(qual_01 ~ `volatile acidity` + `residual sugar` + `sulphates` + `alcohol` + `residual sugar`:`sulphates`, 
                data = wine_train, family = "binomial")
log_fit4 <- glm(qual_01 ~ `volatile acidity`*`residual sugar` + `sulphates` + `alcohol`, 
                data = wine_train, family = "binomial")
log_fit5 <- glm(qual_01 ~ poly(`volatile acidity`, 2) + poly(`sulphates`, 2) + poly(`alcohol`, 2), 
                data = wine_train, family = "binomial")

In [39]:
summary(log_fit2)


Call:
glm(formula = qual_01 ~ `volatile acidity` + `residual sugar` + 
    sulphates + alcohol, family = "binomial", data = wine_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0342  -0.8674   0.3970   0.7738   2.7902  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -11.96738    0.95036 -12.592  < 2e-16 ***
`volatile acidity`  -4.20952    0.49897  -8.436  < 2e-16 ***
`residual sugar`     0.07002    0.01598   4.381 1.18e-05 ***
sulphates            3.16281    0.56277   5.620 1.91e-08 ***
alcohol              1.15603    0.08147  14.190  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1702.3  on 1299  degrees of freedom
Residual deviance: 1289.0  on 1295  degrees of freedom
AIC: 1299

Number of Fisher Scoring iterations: 5


In [43]:
compareFitStats <- function(fit1, fit2){
    require(MuMIn)
    fitStats <- data.frame(fitStat = c("AIC", "AICc"),
                           col1 = AIC(fit1), MuMIn::AICc(fit1), 
                           col2 = AIC(fit2), MuMIn::AICc(fit2))
#put names on returned df
    calls <- as.list(match.call())
    calls[[1]] <- NULL
    names(fitStats)[2:3] <- unlist(calls)
    fitStats
}

In [44]:
compareFitStats(log_fit1, log_fit2)

fitStat,log_fit1,log_fit2,col2,MuMIn..AICc.fit2.
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
AIC,1300.584,1300.649,1298.99,1299.037
AICc,1300.584,1300.649,1298.99,1299.037


In [129]:
?predict

predict {stats},R Documentation
object,a model object for which prediction is desired.
...,additional arguments affecting the predictions produced.
